# Inference from any pose

With this notebook you can run inference of HOOD starting with a garment in any pose. For the body sequence you can use either:
* Sequence of SMPL parameters (see `$HOOD_DATA/fromanypose/pose_sequence.pkl` as an example) or
* Sequence of meshes (see `$HOOD_DATA/fromanypose/mesh_sequence.pkl` as an example)

You also need to have a garment mesh (as `.obj` file) which is aligned with the first frame of your body sequence.

In this notebook we first show how to convert a garment mesh in `.obj` format into a `.pkl` template used in HOOD. 

Then, we show how to use the configuration file `aux/from_any_pose.yaml` to run inference with over albitrary SMPL sequence and arbitrary mesh sequence.

Note that, pinned vertices are not yet supported in this example, so do not expect it to work with lower body garments (they would slide down :c)

## Set environmental variables

In [1]:
import os

HOOD_PROJECT = "/home/agrigorev/Workdir/00_Projects/hood_public"
HOOD_DATA = "/home/agrigorev/Data/01_Projects/hood_public"

os.environ["HOOD_PROJECT"] = HOOD_PROJECT
os.environ["HOOD_DATA"] = HOOD_DATA

In [2]:
%load_ext autoreload
%autoreload 1
%aimport utils.mesh_creation
%aimport datasets.from_any_pose

## Create template file from the .obj file:

Use `utils.mesh_creation::obj2template()` function to convert an `.obj` file into a template dictionary and then save it with `pickle_dump`

`$HOOD_DATA/fromanypose/tshirt.obj` is provided as an example

In [8]:
import trimesh
from utils.mesh_creation import obj2template
from utils.common import pickle_load, pickle_dump
from utils.defaults import DEFAULTS
from pathlib import Path

In [10]:
obj_path = Path(DEFAULTS.data_root) / 'fromanypose' / 'tshirt.obj'
out_template_path = Path(DEFAULTS.data_root) / 'fromanypose' / 'tshirt.pkl'

template_dict = obj2template(obj_path, verbose=True)
pickle_dump(template_dict, out_template_path)

Adding coarse edges... (may take a while)
Done.


## Inference with a SMPL sequence

Here we use a sequence of SMPL parameters as a body sequence. The garment template needs to be aligned with the first frame of the sequence.

The SMPL pose sequence has to be a `.pkl` file containing a dictionary of the following items (see `$HOOD_DATA/fromanypose/pose_sequence.pkl` as an example):
* `body_pose`: np.array [N, 69]
* `global_orient`: np.array [N, 3]
* `transl`: np.array [N, 3]
* `betas`: np.array [10,]

### Load runner from `from_any_pose` config

In [3]:
from utils.validation import Config as ValidationConfig
from utils.arguments import load_params, create_modules
from utils.arguments import load_params
from utils.common import move2device, pickle_dump
from utils.defaults import DEFAULTS
from pathlib import Path
import torch
from utils.arguments import create_runner

In [4]:
modules, config = load_params('aux/from_any_pose')

runner_module, runner, aux_modules = create_runner(modules, config)

checkpoint_path = Path(DEFAULTS.data_root) / 'trained_models' / 'postcvpr.pth'
state_dict =  torch.load(checkpoint_path)
runner.load_state_dict(state_dict['training_module'])

<All keys matched successfully>

### Create a data loader

To do that, first, set several variables to specify which garment and which pose sequence you want to use:

- `pose_sequence_type`: type of the pose sequence can be either `smpl` or `mesh`. For this example, set `smpl`
- `pose_sequence_path` path to the smpl pose sequence relative to `$HOOD_DATA`. For this example we use `fromanypose/pose_sequence.pkl`
- `garment_template_path` path to the .pkl file with the garment template  For this example we use `fromanypose/tshirt.pkl`
- `smpl_model` path to the SMPL model you want to use relative to `$HOOD_DATA`.  For this example we use `smpl/SMPL_FEMALE.pkl`


Then, we create a dataloader with `make_fromanypose_dataloader()` function (see its code for more details)

In [5]:
from utils.datasets import make_fromanypose_dataloader

pose_sequence_type = 'smpl'
pose_sequence_path = Path('fromanypose') / 'pose_sequence.pkl'
garment_template_path = Path('fromanypose') / 'tshirt.pkl'
smpl_model = 'smpl/SMPL_FEMALE.pkl'

dataloader = make_fromanypose_dataloader(pose_sequence_type=pose_sequence_type, 
                       pose_sequence_path=pose_sequence_path, 
                       garment_template_path=garment_template_path, 
                       smpl_model=smpl_model)

### load sample, infer and save trajectories

To visualise saved trajectories, see `Inference.ipynb::write_video`

In [6]:
sample = next(iter(dataloader))

trajectories_dict = runner.valid_rollout(sample)
# Save the sequence to disc
out_path = Path(DEFAULTS.data_root) / 'temp' / 'output_f.pkl'
print(f"Rollout saved into {out_path}")
pickle_dump(dict(trajectories_dict), out_path)

100%|██████████| 171/171 [00:06<00:00, 26.39it/s]

Rollout saved into /home/agrigorev/Data/01_Projects/hood_public/temp/output_f.pkl


## Inference with a mesh sequence

Here we use a sequence of arbitrary meshes.  The garment template needs to be aligned with the first frame of the sequence.

The mesh sequence has to be a `.pkl` file containing a dictionary of the following items (see `$HOOD_DATA/fromanypose/mesh_sequence.pkl` as an example):
* `verts`: np.array [N, 3]
* `faces`: np.array [F, 3]

### Load runner and dataloader from `from_any_pose` config

In [7]:
from utils.validation import Config as ValidationConfig
from utils.arguments import load_params, create_modules
from utils.arguments import load_params
from utils.common import move2device, pickle_dump
from utils.defaults import DEFAULTS
from pathlib import Path
import torch

In [8]:
modules, config = load_params('aux/from_any_pose')


runner_module, runner, aux_modules = create_runner(modules, config)

checkpoint_path = Path(DEFAULTS.data_root) / 'trained_models' / 'postcvpr.pth'
state_dict =  torch.load(checkpoint_path)
runner.load_state_dict(state_dict['training_module'])

<All keys matched successfully>

### Create a data loader

Same as in the previous part, set several variables to specify which garment and which pose sequence you want to use:

- `pose_sequence_type`: type of the pose sequence can be either `smpl` or `mesh`. For this example, set `mesh`
- `pose_sequence_path` path to the smpl pose sequence relative to `$HOOD_DATA`. For this example we use `fromanypose/mesh_sequence.pkl`
- `garment_template_path` path to the .pkl file with the garment template  For this example we use `fromanypose/tshirt.pkl`

In [9]:
pose_sequence_type = 'mesh'
pose_sequence_path = Path('fromanypose') / 'mesh_sequence.pkl'
garment_template_path = Path('fromanypose') / 'tshirt.pkl'


dataloader = make_fromanypose_dataloader(pose_sequence_type=pose_sequence_type, 
                       pose_sequence_path=pose_sequence_path, 
                       garment_template_path=garment_template_path)

### load sample, infer and save trajectories

To visualise saved trajectories, see `Inference.ipynb::write_video`

In [11]:
sample = next(iter(dataloader))

trajectories_dict = runner.valid_rollout(sample)
# Save the sequence to disc
out_path = Path(DEFAULTS.data_root) / 'temp' / 'output_m.pkl'
print(f"Rollout saved into {out_path}")
pickle_dump(dict(trajectories_dict), out_path)

100%|██████████| 171/171 [00:06<00:00, 28.08it/s]

Rollout saved into /home/agrigorev/Data/01_Projects/hood_public/temp/output_m.pkl
